# SQLAlchemy Testing

```bash
# Launch MySQL server with mod_akrr database
docker run -it --rm --name akrr -h akrr \
    -v /home/nikolays/xdmod_wsp/access_akrr/mysql:/var/lib/mysql \
    -v /home/nikolays/xdmod_wsp/access_akrr/akrr/akrr_home:/home/akrruser/akrr \
    -p 3370:3306 -p 2270:22 \
    nsimakov/akrr_ready_centos_slurm_single_host_wlm:latest cmd_start sshd mysqld bash

# Rocky 8 MariaDB 10.3
# Launch MySQL server with mod_akrr database
docker run -it --rm --name akrr3 -h akrr3 \
    -v /home/nikolays/xdmod_wsp/access_akrr/mysql_akrr3_test:/var/lib/mysql \
    -v /home/nikolays/xdmod_wsp/access_akrr/akrr/akrr_home:/home/akrruser/akrr \
    -v /home/nikolays/xdmod_wsp/akrr:/home/akrruser/akrr_src \
    -p 3371:3306 -p 2271:22 \
    nsimakov/akrr_ready:latest cmd_start sshd mysqld bash
```

In [1]:
%load_ext autoreload
%autoreload 2
from update2to3 import *

DEBUG:root:AKRR_HOME is in default location. AKRR configuration is in /home/nikolays/akrr/etc/akrr.conf
DEBUG:root:AKRR_HOME is in default location. AKRR configuration is in /home/nikolays/akrr/etc/akrr.conf
DEBUG:root:Loading app test
DEBUG:root:Loading app gamess
DEBUG:root:Loading app gamess
DEBUG:root:Loading app gamess
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app nwchem
DEBUG:root:Loading app bundle
DEBUG:root:Loading app template
DEBUG:root:Loading app gromacs
DEBUG:root:Loading app gromacs
DEBUG:root:Loading app gromacs
DEBUG:root:Loading app gromacs
DEBUG:root:Loading app gromacs
DEBUG:root:Loading app graph500
DEBUG:root:Loading app graph500
DEBUG:root:Loading app graph500
DEBUG:root:Loading app graph500
DEBUG:root:Loading app graph500
DEBUG:root:Loading app graph500
DEBUG:root:Loading app gr

In [2]:
# extra config for update
from akrr.util import exec_files_to_dict
update_param = exec_files_to_dict("update_param.py")
resource_ppn_lookup = update_param['resource_ppn_lookup']
# call get_ppn with (resource_ppn_lookup)

## Make a Model

In [17]:
reset_new_db()

In [18]:
import models

In [19]:
with engine_appkernel.begin() as connection:
    models.mapper_registry.metadata.create_all(connection)

# Get Values From DB

In [20]:
with engine_akrr.connect() as connection:

    result = connection.execute(text('''
        select instance_id as task_id, resource, reporter as app, collected,
            committed, executionhost,  reporternickname, status, message,
            stderr, body, memory, cputime, walltime, job_id, internal_failure,
            nodes, ncores, nnodes
        from akrr_xdmod_instanceinfo where status=1 and reporter="namd" limit 3'''))
    tasks=result.fetchall()

with engine_appkernel_v2.connect() as connection:
    stmt = text('''
SELECT `resource`.`resource_id`,
    `resource`.`resource` as name,
    `resource`.`nickname` as short_name,
    `resource`.`description`,
    `resource`.`enabled`,
    `resource`.`visible`,
    `resource`.`xdmod_resource_id`,
    `resource`.`xdmod_cluster_id`
FROM `mod_appkernel`.`resource`''')
    result = connection.execute(stmt)
    resource_mod_appkernel = result.fetchall()


In [21]:
root_elm = ElementTree.fromstring(tasks[0].body)
benchmark_name = root_elm.find(".//benchmark/ID").text
params_elm = root_elm.find(".//parameters")
stats_elm = root_elm.find(".//statistics")

print(f'Benchmark: {benchmark_name}')
for param in params_elm:
    name = param.find("ID").text
    value = param.find("value").text
    units = param.find("units")
    if units is not None:
        units = units.text
    print(f'\t{name} {value} {units}')

for stat in stats_elm:
    name = stat.find("ID").text
    value = stat.find("value").text
    units = stat.find("units")
    if units is not None:
        units = units.text
    guid = models.guid_calc(name+units)
    print(f'\t{name} {value} {units} {guid}')

Benchmark: NAMD
	App:ExeBinSignature H4sIAB6RGVECA4WQy27bMBBF9/4KLtO6lTlDio/sajkNDDhuYKMBujL4GLYC/IIkF83fl8rDUdoGhQAuxHPPzGX15eZ2vrhaXbLrqmKykMUvdoGsq3fUfmBaMH/fUftuVP3Fif9wt9dz9vnQdI3bs+qwO9Zbaph6pRfmLT0UyC7gmQP7FlcW/J/c7NtyMZ9malv7wLAAfPm1/Mqq8ZitO7eProlsUfvGNfdM5FIgXrg8YHXa92q2Ph2PucoZze0Lfibn+1Tv62nWsTtqfHvGoIDRzay8ZBJCMl5Ja0tjjFWoEVTwSVkEEzSw95M6tZNTV28fDiUne7eLk+Wnm9kGC7NZH05NoI/z6d3Varp+uMRHNU/JaNV/XnKuE4/RcofeixhFFCmrT20zye+Qpfms/c9+x6I9PC/HnSwpJc2BhFKpLMFLAZGiFQT5KhvO6bjtg0+jZcRSCksWpRU+RBLBaIqKIAVSwg2Duz6nHnMmWkzJAgEanjQioSInfQpRK1Lmz5XbLobxeCAQzoYgSGh0kmR2OaEDcC08oEKyw8HfQ9gM25KNioNQLnHgKXdXECUlLxElUHqVDYOZsfQJKb+2slqGwKUXQD7oiJrbZNQwd+x+NORin+aj0W8WqescYwMAAA== None
	App:NCores 96 None
	App:NNodes 8 None
	App:Version 2.8 None
	Input:Coordinate File apoa1.pdb None
	Input:Number of Angles 74136 None
	Input:Number of Atoms 92224 None
	Input:Number of Bonds 70660 None
	Input:Number of Dihedrals 74130 None
	Input:Number of Steps 1200 None
	Input:Structure File apoa1.psf None
	In

# Populate

In [ ]:
def MoveResourcesTable(resources, Session):
    """
    add resources to db
    """
    with Session.begin() as session:
        for r in resource_mod_appkernel:
            resource = session.query(Resource).filter_by(name=r.name).one_or_none()
            if resource is not None:
                log.warning("Resource {r.name} already in")
                continue
            resource = Resource(resource_id=r.resource_id, name=r.name, short_name = r.short_name, description=r.description,
                     enabled=r.enabled, visible=r.visible, xdmod_resource_id=r.xdmod_resource_id, xdmod_cluster_id=r.xdmod_cluster_id)
            session.add(resource)
        session.commit()


def IngestTask(task, Session):
    """
    return True on succesfull ingestion
    """
    with Session.begin() as session:
        try:
            resource = session.query(Resource).filter_by(name=task.resource).one()
        except sqlalchemy.orm.exc.NoResultFound:
            try:
                resource = session.query(Resource).filter_by(short_name=task.resource).one()
            except sqlalchemy.orm.exc.NoResultFound:
                log.error(f"Unknown resource({task.resource}) to DB. Add it first.")
                return False

        ak_id = 1
        task_id = task.task_id
        # read params and stats from result.xml
        root_elm = ElementTree.fromstring(task.body)
        root_elm.find("ID").text
        # benchmark_name = root_elm.find(".//benchmark/ID").text
        # params_elm = root_elm.find(".//parameters")
        stats_elm = root_elm.find(".//statistics")


        for stat in stats_elm:
            name = stat.find("ID").text
            value = stat.find("value").text
            units = stat.find("units")
            if units is not None:
                units  = units.text
            guid = guid_calc(name+units)

            # get metric
            try:
                metric = session.query(Metric).filter_by(guid=guid).one()
            except sqlalchemy.orm.exc.NoResultFound:
                metric = Metric(name=name, unit=units, guid=guid, type=models.DataType.float)
                session.add(metric)

            # get metric_val
            try:
                metric_val = session.query(MetricNumData).filter_by(
                    resource=resource, ak_id=ak_id, metric=metric, task_id=task_id).one()
            except sqlalchemy.orm.exc.NoResultFound:
                metric_val = MetricNumData(
                    resource=resource, ak_id=ak_id, metric=metric, task_id=task_id)
                session.add(metric_val)

            # set values
            metric_val.collected=task.collected,
            metric_val.value=value

            # reset control on reingest
            metric_val.running_average=None
            metric_val.control=None
            metric_val.control_region_id=None
            metric_val.control_status=models.ControlStatus.undefined

        session.commit()
    return True

In [ ]:
def create_or_get(session, cl, **kwargs):
    try:
        o = session.query(cl).filter_by(**kwargs).one()
    except sqlalchemy.orm.exc.NoResultFound:
        o = cl(**kwargs)
        session.add(o)
    return o

with SessionAKRR.begin() as session:
    core = create_or_get(session, ComputeResourceType, name="core")
    node = create_or_get(session, ComputeResourceType, name="node")
    gpu = create_or_get(session, ComputeResourceType, name="GPU")
    node1 = create_or_get(session, ComputeResourceSpec, compute_resource_type=node, proc_units=1)
    node2 = create_or_get(session, ComputeResourceSpec, compute_resource_type=node, proc_units=2)
    node4 = create_or_get(session, ComputeResourceSpec, compute_resource_type=node, proc_units=4)
    node8 = create_or_get(session, ComputeResourceSpec, compute_resource_type=node, proc_units=8)
    node16 = create_or_get(session, ComputeResourceSpec, compute_resource_type=node, proc_units=16)
    core8 = create_or_get(session, ComputeResourceSpec, compute_resource_type=core,proc_units=8)
    core16 = create_or_get(session, ComputeResourceSpec, compute_resource_type=core,proc_units=16)
    core32 = create_or_get(session, ComputeResourceSpec, compute_resource_type=core,proc_units=32)
    core64 = create_or_get(session, ComputeResourceSpec, compute_resource_type=core,proc_units=64)
    core128 = create_or_get(session, ComputeResourceSpec, compute_resource_type=core,proc_units=128)

    node1_core8 = TotalComputeResourceSpec()
    session.add(node1_core8)
    session.add(TotalComputeResourceSpecList(total_compute_resource_spec=node1_core8, compute_resource_spec=node1))
    session.add(TotalComputeResourceSpecList(total_compute_resource_spec=node1_core8, compute_resource_spec=core8))
    #node1_core8 = create_or_get(session, TotalComputeResourceSpec, id=1)
    #node2_core16 = create_or_get(session, TotalComputeResourceSpec, id=2)
    session.commit()



In [ ]:
spec = session.query(TotalComputeResourceSpec).all()
spec

In [ ]:
spec[0].total_compute_resource_spec_list

In [ ]:
MoveResourcesTable(resource_mod_appkernel, Session)

for task in tasks:
    IngestTask(task, Session)

In [ ]:
session = Session()
select_statement = select(Resource)
result = session.execute(select_statement)

In [ ]:
result.fetchall()

In [ ]:
from models_v2 import Resource as ResourceOld
with SessionModAppKernel.begin() as session:
    session.expire_on_commit=False
    resources = session.query(ResourceOld).all()
    print(resources)
    r1 = resources[0]
    print(r1.nickname)

In [ ]:
r1 = resources[0]

In [ ]:
r1.nickname

In [ ]:


with SessionModAppKernel() as session:
    resources_old_appkernel = session.query(ResourceModAKOld).all()
s=""
for r in resources_old_appkernel:
    ppn1 = get_ppn(r.nickname,ppn_lookup=resource_ppn_lookup,return_all_ppn=False)
    ppn2 = get_ppn(r.resource,ppn_lookup=resource_ppn_lookup,return_all_ppn=False)
    if ppn1 is None and ppn2 is not None:
        ppn1=ppn2
    s=s+f"    '{r.nickname}': {ppn1},  # {r.resource}, {ppn2}\n"
print(s)

In [ ]:
import akrr
from akrr.cfg import find_resource_by_name

In [ ]:
try:
    resource = find_resource_by_name("anvil")
except akrr.AkrrError:
    resource =


In [ ]:
"""SELECT `resourcefact`.`id` as resource_id,
    `resourcefact`.`resourcetype_id`,
    `resourcefact`.`organization_id`,
    `resourcefact`.`name`,
    `resourcespecs`.`name` as name2,
    `resourcefact`.`code`,
	`resourcespecs`.`processors`,
    `resourcespecs`.`q_nodes`,
    `resourcespecs`.`q_ppn`,
    `resourcefact`.`description`,
	`resourcespecs`.`comments`,
    `resourcefact`.`start_date`,
    `resourcefact`.`end_date`,
    `resourcefact`.`shared_jobs`,
    `resourcefact`.`timezone`,
    `resourcefact`.`resource_origin_id`,
    `resourcespecs`.`processors`,
    `resourcespecs`.`q_nodes`,
    `resourcespecs`.`q_ppn`
FROM `modw`.`resourcespecs`
LEFT JOIN `modw`.`resourcefact` ON `resourcefact`.`id`=`resourcespecs`.`resource_id`;"""